### Import modules

In [1]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
import warnings
warnings.filterwarnings("ignore")

# NYCOD

## Import

In [3]:
import pandas as pd
import numpy as np

#read in sales data, already subsetted to include Manhattan only
sales17 = pd.read_csv("INPUT_nycod/2017_manhattan.csv", skiprows=5, header=None).dropna(how='all')
sales18 = pd.read_csv("INPUT_nycod/2018_manhattan.csv", skiprows=5, header=None).dropna(how='all')
sales19 = pd.read_csv("INPUT_nycod/2019_manhattan.csv", skiprows=5, header=None).dropna(how='all')
sales20 = pd.read_csv("INPUT_nycod/2020_manhattan.csv", skiprows=7, header=None).dropna(how='all')
sales21 = pd.read_csv("INPUT_nycod/2021_manhattan.csv", skiprows=7, header=None).dropna(how='all')
sales22 = pd.read_csv("INPUT_nycod/rollingsales_manhattan.csv", skiprows=1, header=None).dropna(how='all')

#concatenate all dfs, check shape
allsales = pd.concat((sales17, sales18, sales19, sales20, sales21, sales22), axis=0)
allsales = allsales.iloc[:,0:21]
print(allsales.shape)

#add labels
labels = pd.DataFrame(pd.read_csv("INPUT_nycod/2021_manhattan.csv", header=6).columns).T
allsales = pd.concat((labels, allsales), axis=0)
allsales.columns = allsales.iloc[0] 
allsales = allsales[1:]

(110226, 21)


## Clean

### Dtype handling

In [4]:
#convert objects to numeric where possible
allsales['BOROUGH'] = allsales['BOROUGH'].astype(np.int64)
allsales['BLOCK'] = allsales['BLOCK'].astype(np.int64)
allsales['LOT'] = allsales['LOT'].astype(np.int64)
allsales['RESIDENTIAL\nUNITS'] = pd.to_numeric(allsales['RESIDENTIAL\nUNITS'].str.replace(',',''), errors='coerce')
allsales['TOTAL \nUNITS'] = pd.to_numeric(allsales['TOTAL \nUNITS'].str.replace(',',''), errors='coerce')
allsales['LAND \nSQUARE FEET'] = pd.to_numeric(allsales['LAND \nSQUARE FEET'].str.replace(',',''), errors='coerce')
allsales['GROSS \nSQUARE FEET'] = pd.to_numeric(allsales['GROSS \nSQUARE FEET'].str.replace(',',''), errors='coerce')
allsales['COMMERCIAL\nUNITS'] = pd.to_numeric(allsales['COMMERCIAL\nUNITS'], errors='coerce')
allsales['YEAR BUILT'] = pd.to_numeric(allsales['YEAR BUILT'], errors='coerce')

#clean sale price
prices = []
for i in allsales['SALE PRICE']:
    prices.append(float(i.replace(",", "").replace("'", "").replace("$", "")))
allsales['saleprice'] = prices
allsales = allsales.drop(columns=['SALE PRICE'])

#clean sale date, create 'years old at time of sale' variable
import datetime
dates = []
for i in allsales['SALE DATE']:
    dates.append(datetime.datetime.strptime(i, "%m/%d/%Y"))
allsales['saledate'] = dates
                 
years = []
for i in allsales['SALE DATE']:
    years.append(int(i[-4:]))
allsales['year_clean'] = years
allsales['bldg_age_at_sale_calc'] = allsales.year_clean - allsales['YEAR BUILT']
allsales = allsales.drop(columns=['YEAR BUILT'])
allsales = allsales.drop(columns=['SALE DATE'])
allsales = allsales.drop(columns=['year_clean'])

### Drop bad columns

In [5]:
#drop irrelevant and duplicate columns
allsales = allsales.drop(columns=['NEIGHBORHOOD',
                                  'BOROUGH', 
                                  'BUILDING CLASS CATEGORY', 
                                  'TAX CLASS AT PRESENT', 
                                  'BUILDING CLASS AT PRESENT',
                                  'TAX CLASS AT TIME OF SALE',
                                  'BUILDING CLASS\nAT TIME OF SALE',
                                  'EASE-MENT'])

#rename columns for interpretability
col_mapper = {'RESIDENTIAL\nUNITS':'units_residential',
             'COMMERCIAL\nUNITS':'units_commercial',
             'TOTAL \nUNITS':'units_total',
             'LAND \nSQUARE FEET': 'area_land',
             'GROSS \nSQUARE FEET': 'area_gross'}

allsales = allsales.rename(mapper=col_mapper, axis=1)

In [6]:
#lowercase and rename
allsales.columns = map(str.lower, allsales.columns)
allsales.columns = [i.replace('\n', " ").replace("  ", " ").replace(" ","") for i in allsales.columns]

allsales = allsales.drop(columns=['area_gross',
                     'area_land',
                     'units_total',
                     'apartmentnumber'])

### Drop bad rows (sale price == 0)

In [ ]:
#FOR VIZ ONLY
#missing sale price
print("total observations: {}".format(len(allsales)))
print("non-sale transfers (sale price 0): {}".format(len(allsales[allsales['saleprice'] == 0])))
print("sales (sale price > 0): {}".format(len(allsales[allsales['saleprice'] > 0])))

import matplotlib.pyplot as plt
plt.hist(allsales[allsales.saleprice > 0].saledate, alpha=0.5, color='lightblue', label='sales', bins=6)
plt.hist(allsales[allsales.saleprice == 0].saledate, alpha=0.5, color='darkblue', label='non-sale transfers', bins=6)
plt.legend()
plt.title('Frequency of property sales vs. non-sale transfers')
plt.xlabel('Transaction year')
plt.ylabel('Number of transactions')
plt.savefig('OUTPUT_visualizations/1.Sales vs. non-sale transfers.png')
plt.show()

In [7]:
#SUBSET DATA
allsales_use = allsales[allsales['saleprice'] > 0]

#after removing zeros
allsales_use['log_saleprice'] = np.log(allsales_use.saleprice)
allsales_use = allsales_use.drop(columns=['saleprice'])

# PLUTO

## Import

In [8]:
pluto = pd.read_csv("INPUT_pluto/pluto_22v3_1.csv", low_memory=False)
pluto = pluto.dropna(how='all')

#subset to include Manhattan only
man = pluto[pluto['borough']=='MN']
print(man.shape)

(42695, 92)


## Clean

### Dtype handling

In [9]:
#make list of PLUTO categorical columns
cate_cols = pd.DataFrame(man.dtypes)[pd.DataFrame(man.dtypes)[0] == 'object']

#create binary variables: ltdheight, splitzone, histdist, landmark
man['bin_ltdheight'] = abs(1-(man.ltdheight.isna()))
man['bin_splitzone'] = abs(1-(man.splitzone.isna()))
man['bin_histdist'] = abs(1-(man.histdist.isna()))
man['bin_landmark'] = abs(1-(man.landmark.isna()))

#drop all categoricals (including the ones we just recoded) but leave address to use for later recodes
cate_cols = cate_cols.drop('address')
man = man.drop(columns=cate_cols.index) 

print(man.shape)

(42695, 69)


### Add columns

In [10]:
#create FAR variable
man['far_calc'] = man['bldgarea'] / man['lotarea']
man = man.drop(columns=['bldgarea', 'lotarea'])

### Drop bad columns

In [11]:
#drop other irrelevant columns
man = man.drop(columns=['notes', 
                        'firm07_flag', 
                        'pfirm15_flag', 
                        'bctcb2020', 
                        'ct2010', 
                        'cb2010'])

In [12]:
#lowercase and rename
man.columns = map(str.lower, man.columns)
man.columns = [i.replace('\n', " ").replace("  ", " ").replace(" ","") for i in man.columns]

man = man.drop(columns=['xcoord',
                     'ycoord',
                     'borocode',
                     'plutomapid',
                     'sanitboro',
                     'unitsres',
                     'zipcode',
                     'areasource',
                     'bbl',
                     'builtfar',
                     'commfar',
                      'cd',
                     'facilfar',
                     'proxcode',
                     'residfar',
                     'taxmap',
                     'unitstotal',
                     'spdist3',
                     'landuse',
                     'easements',
                     'bsmtcode',
                     'assessland',
                     'assesstot',
                     'exempttot',
                     'condono',
                     'lottype',
                     'lotfront',
                     'lotdepth',
                     'bldgfront',
                     'bldgdepth',
                     'appbbl',
                     'tract2010'])

In [ ]:
man.columns

In [ ]:
#corr matrix of the district variables
import matplotlib.pyplot as plt
f = plt.figure(figsize=(14, 10))
plt.matshow(man[['policeprct', 'schooldist', 'healthcenterdistrict', 'sanitdistrict', 'council']].corr(), fignum=f.number)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=10)
plt.xticks(range(0, 5, 1), ['policeprct', 'schooldist', 'healthcenterdistrict', 'sanitdistrict', 'council'], fontsize=10)
plt.yticks(range(0, 5, 1), ['policeprct', 'schooldist', 'healthcenterdistrict', 'sanitdistrict', 'council'], fontsize=10)
plt.title('Correlation between locational variables')
plt.savefig('OUTPUT_visualizations/2.District overlap.png')
plt.show()

In [13]:
binary_cols = ['policeprct','schooldist','healthcenterdistrict', 'sanitdistrict', 'council']
for i in binary_cols:
    man[i] = man[i].astype(object)
    
man.dtypes

block                     int64
lot                       int64
bct2020                 float64
schooldist               object
council                  object
policeprct               object
healthcenterdistrict     object
healtharea              float64
sanitdistrict            object
address                  object
comarea                 float64
resarea                 float64
officearea              float64
retailarea              float64
garagearea              float64
strgearea               float64
factryarea              float64
otherarea               float64
numbldgs                float64
numfloors               float64
yearbuilt               float64
yearalter1              float64
yearalter2              float64
latitude                float64
longitude               float64
bin_ltdheight             int32
bin_splitzone             int32
bin_histdist              int32
bin_landmark              int32
far_calc                float64
dtype: object

In [14]:
man = man[man.block < man.block.max()]
print(man.shape)

(42694, 30)


# Merge NYCOD and PLUTO

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(man.block, man.lot, label='Included in PLUTO', color='blue', marker='o')
plt.scatter(allsales_use.block, allsales_use.lot, label='Actual sales (NYCOD)', color='lightgray', alpha=0.1, marker='.')
plt.xlabel('Block code')
plt.ylabel('Lot code')
plt.title('Block-Lot codes covered by PLUTO')
plt.legend()
plt.savefig('OUTPUT_visualizations/3.Borough-Lot coverage.png')
plt.show()

In [15]:
for i in allsales_use.columns:
    if i in man.columns:
        print(i)
        
man = man.rename(mapper={'block':'block_pluto',
                        'lot':'lot_pluto',
                        'address':'address_pluto'}, axis=1)

block
lot
address


In [16]:
#join based on BBL

df_1 = allsales_use.merge(man, how='left', left_on=['block', 'lot'], right_on=['block_pluto', 'lot_pluto'])
print('Total transactions (from NYCOD): {}'.format(len(df_1)))
print("Successfully mapped to PLUTO via BBL: {:.3f}%".format(100*(len(df_1) - len(df_1[df_1.latitude.isna() == True]))/(len(df_1))))

Total transactions (from NYCOD): 88439
Successfully mapped to PLUTO via BBL: 49.474%


In [17]:
#join based on address

#isolate transactions that were not matched before
missed = df_1[df_1['latitude'].isna() == True].iloc[:,:10]

#clean address text
addresses = []
for i in missed.address:
    addresses.append(i.split(',')[0])
missed['address_clean'] = addresses

#merge on address
df_2 = missed.merge(man, how='left', left_on=['address_clean'], right_on=['address_pluto'])

print('Unmapped transactions (from NYCOD): {}'.format(len(df_2)))
print("Successfully mapped to PLUTO via address: {:.3f}%".format(100*(len(df_2) - len(df_2[df_2.latitude.isna() == True]))/(len(df_2))))

Unmapped transactions (from NYCOD): 49458
Successfully mapped to PLUTO via address: 37.420%


In [20]:
len_all = len(allsales_use)
len_bbl = len(df_1)
len_add = len(df_2)

df_1 = df_1[df_1.latitude.isna() == False].drop(columns=['address_pluto'])
df_2 = df_2[df_2.latitude.isna() == False].drop(columns=['address_clean', 'address_pluto'])

print('{} total sales'.format(len_all))
print('{} matched on BBL'.format(len_bbl))
print('{} matched on address'.format(len_add))
print('{} unmatched ({:.3f}% of total)'.format((len_all - len_bbl - len_add), 100*(len_all - len_bbl - len_add) / len_all))

df = pd.concat((df_1, df_2), axis=0)
#drop empty
df = df.dropna(how='all', axis=1)
df = df.dropna(how='all', axis=0)
#final df
print('\n')
print('shape of final df: {}'.format(df.shape))
df.head()

df.to_csv('OUTPUT_cleandata/df_clean.csv')

KeyError: "['address_pluto'] not found in axis"

In [ ]:
# plot corr matrix of all vars

f = plt.figure(figsize=(14, 8))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=10, rotation=90)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=10)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=10)
plt.title('Correlation between independent variables')
plt.savefig('OUTPUT_visualizations/4.Corr matrix for all vars.png')
plt.show()

In [19]:
#drop cols with more than 90% missing data
cols = pd.DataFrame(df.columns).rename(columns={0:'column_name'})
missing = []
for i in cols['column_name']:
    missing.append(100*df[i].isnull().sum()/len(df[i]))
cols['pct_missing'] = missing
to_drop = []
for i in cols.index:
    if cols['pct_missing'][i] > 90:
        to_drop.append(cols['column_name'][i])
df = df.drop(columns=to_drop)

print('Features with > 90% missing data: {} \n'.format(to_drop))
print('Features with any missing data:')
display(cols[cols['pct_missing']>0].sort_values('pct_missing', ascending=False))

Features with > 90% missing data: ['block_pluto_x'] 

Features with any missing data:


,column_name,pct_missing
38,block_pluto_x,99.950210
39,block_pluto_y,70.275132
4,units_residential,68.607957
5,units_commercial,58.171247
9,block_pluto,29.724868
27,numfloors,4.616052
7,bldg_age_at_sale_calc,3.954321
37,far_calc,3.596152
26,numbldgs,3.498177
24,factryarea,1.034355


In [ ]:
df.saledate

In [ ]:
#plot hist of sale prices

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.hist(df['log_saleprice'])
plt.xlabel('Ln of sale price ($)')
plt.ylabel('Frequency')
plt.title('Manhattan residential real estate sales, 2017-2022')
plt.savefig('OUTPUT_visualizations/5.Histogram of ln sale price.png')
plt.show()

In [ ]:
#'bct2020_pluto' is 2020 census tract if needed for tabulation

# Models

# Create four different preprocessors

### Sort chronologically to train test split

In [ ]:
#train test split

#sort values by date
df = df.sort_values('saledate')
df = df.drop(columns=['saledate'])

# drop sale price

#subset into X and y
X = df.drop('log_saleprice', axis=1)
y = df['log_saleprice']

#split into train test split
train_size = round(len(df)*0.75)
X_train = X[:train_size]
X_test = X[train_size:]
y_train = y[:train_size]
y_test = y[train_size:]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## RF preprocessor, full model

In [ ]:
X_train.dtypes

X_train = X_train.drop(columns=['address', 'zipcode'])

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_features_full = X_train.select_dtypes([np.number]).columns.tolist()
categorical_features_full = X_train.select_dtypes([object]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

preprocessor_full_rf = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_full),
        ('cat', categorical_transformer, categorical_features_full)])

preprocess_full_rf = preprocessor_full_rf.fit(X_train) 

def preprocessor_full_rf(data):
    preprocessed_data = preprocess_full_rf.transform(data)
    return preprocessed_data

## OLS preprocessor, full model

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#turn columns into categorical variables: zip code, police precinct, school district, etc.
#ENSURE COLS ARE OHE: ['policeprct', 'schooldist', 'healthcenterdistrict', 'sanitdistrict', 'council']

numeric_features_full = X_train.select_dtypes([np.number]).columns.tolist()

categorical_features_full = X_train.select_dtypes([object]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor_full_ols = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_full),
        ('cat', categorical_transformer, categorical_features_full)])

preprocessor_full_ols = preprocessor_full_ols.fit(X_train) 

def preprocessor_full_ols(data):
    preprocessed_data=preprocessor_full_ols.transform(data)
    return preprocessed_data

## RF preprocessor, subset

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_features_sub = X_train_imp.select_dtypes([np.number]).columns.tolist()
categorical_features_sub = X_train_imp.select_dtypes([object]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

preprocessor_sub_rf = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_full),
        ('cat', categorical_transformer, categorical_features_full)])

preprocess_sub_rf = preprocessor_sub_rf.fit(X_train_imp) 

def preprocessor_sub_rf(data):
    preprocessed_data = preprocess_sub_rf.transform(data)
    return preprocessed_data

## OLS preprocessor, subset

In [ ]:
numeric_features_sub = X_train_imp.select_dtypes([np.number]).columns.tolist()
categorical_features_sub = X_train_imp.select_dtypes([object]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocess_sub_ols = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_sub),
        ('cat', categorical_transformer, categorical_features_sub)])

preprocess_sub_ols = preprocess_sub_ols.fit(X_train_imp) 

def preprocessor_sub_ols(data):
    preprocessed_data = preprocess_sub_ols.transform(data)
    return preprocessed_data

# Run and compare models

## RF, all features

In [ ]:
## Random Forest for feat. selection

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from statistics import mean
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'max_depth': [10, 20, 30, 40, 50],
              'n_estimators': [10, 20, 30, 40, 50]}

grid = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, cv=10)

grid.fit(preprocessor_full_rf(X_train), y_train)

print("best median cross-val score: {}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))
print("test-set score: {}".format(grid.score(preprocessor_full_rf(X_test), y_test)))

In [ ]:
best_forest = RandomForestRegressor(max_depth = grid.best_params_['max_depth'], n_estimators=grid.best_params_['n_estimators']).fit(preprocessor_full_rf(X_train), y_train)

forest_1_predictions = best_forest.predict(preprocessor_full_rf(X_test))

feature_names = X_train.columns
forest_importances = pd.DataFrame(best_forest.feature_importances_, index=feature_names).sort_values([0], ascending=False)

plt.figure(figsize=(12, 8))
plt.bar(forest_importances.index, forest_importances[0])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature importances from best Random Forest model; max depth {} and n_estimators {}'.format(
    grid.best_params_['max_depth'], grid.best_params_['n_estimators']))
plt.grid()
plt.savefig('OUTPUT_models/featureimportances.png')
plt.show()

In [ ]:
most_imp_cols = forest_importances.index[:20]

X_train_imp = X_train[most_imp_cols]
X_test_imp = X_test[most_imp_cols]

## RF, subset

In [ ]:
rf = RandomForestRegressor(n_estimators=20, max_depth=20).fit(preprocessor_sub_rf(X_train_imp), y_train)
#param_grid = {'max_depth': [10, 15, 20, 25],
              #'n_estimators': [10, 30, 50, 100]}
#grid = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, cv=10)

print("Train set score: {:.5f}".format(rf.score(preprocessor_sub_rf(X_train_imp), y_train)))
print("Mean cross-val score for train set: {:.5f}".format(cross_val_score(rf, preprocessor_sub_rf(X_train_imp), y_train, scoring='r2').mean()))
print("Test set score: {:.5f}.".format(rf.score(preprocessor_sub_rf(X_test_imp), y_test)))

forest_2_predictions = rf.predict(preprocessor_sub_rf(X_test_imp))

## OLS, all features

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from statistics import mean
from sklearn.metrics import mean_squared_error

lr = LinearRegression().fit(preprocessor_full_ols(X_train), y_train)

print("Train set score: {:.5f}".format(lr.score(preprocessor_full_ols(X_train), y_train)))
print("Mean cross-val score for train set: {:.5f}".format(cross_val_score(lr, preprocessor_full_ols(X_train), y_train, scoring='r2').mean()))
print("Test set score: {:.5f}.".format(lr.score(preprocessor_full_ols(X_test), y_test)))

ols_1_predictions = lr.predict(preprocessor_full_ols(X_test))

#sort results by coefficient size
ols_1_results = pd.concat((pd.DataFrame(X_train.columns), pd.DataFrame(lr.coef_)), axis=1)
ols_1_results.columns = ['column', 'coef']
ols_1_results = ols_1_results.sort_values('coef', ascending=True)

#plot figure
plt.figure(figsize=(20, 5))
plt.bar(ols_1_results.column, ols_1_results.coef)
plt.xticks(rotation=90)
plt.grid()
plt.xlabel("Feature")
plt.ylabel('Coefficient size')
plt.savefig("OUTPUT_models/fullolscoefs.png")
plt.show()

## OLS, subset

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from statistics import mean
from sklearn.metrics import mean_squared_error

lr = LinearRegression().fit(preprocessor_sub_ols(X_train_imp), y_train)

print("Train set score: {:.5f}".format(lr.score(preprocessor_sub_ols(X_train_imp), y_train)))
print("Mean cross-val score for train set: {:.5f}".format(cross_val_score(lr, preprocessor_sub_ols(X_train_imp), y_train, scoring='r2').mean()))
print("Test set score: {:.5f}.".format(lr.score(preprocessor_sub_ols(X_test_imp), y_test)))

ols_2_predictions = lr.predict(preprocessor_sub_ols(X_test_imp))

#sort results by coefficient size
ols_2_results = pd.concat((pd.DataFrame(X_train_imp.columns), pd.DataFrame(lr.coef_)), axis=1)
ols_2_results.columns = ['column', 'coef']
ols_2_results = ols_2_results.sort_values('coef', ascending=True)

#plot figure
plt.figure(figsize=(20, 5))
plt.bar(ols_2_results.column, ols_2_results.coef)
plt.xticks(rotation=90)
plt.grid()
plt.xlabel("Feature")
plt.ylabel('Coefficient size')
plt.savefig("OUTPUT_models/subsetcoefs.png")
plt.show()

## Compare models

In [ ]:
results = pd.DataFrame()
results['Model'] = ['RF, all features', 'RF, 20 features', 'OLS, all features', 'OLS, 20 features']
results = results.set_index('Model')

from sklearn.metrics import mean_squared_error
import math

#RMSE AND %RMSE
results['RMSE'] = [math.sqrt(mean_squared_error(forest_1_predictions, y_test)), 
                  math.sqrt(mean_squared_error(forest_2_predictions, y_test)), 
                  math.sqrt(mean_squared_error(ols_1_predictions, y_test)), 
                  math.sqrt(mean_squared_error(ols_2_predictions, y_test))]

def percent_rmse(y_true, y_pred):
    rmspe = np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true)), axis=0))
    return rmspe
results['%RMSE'] = [percent_rmse(y_test, forest_1_predictions),
                   percent_rmse(y_test, forest_2_predictions),
                   percent_rmse(y_test, ols_1_predictions),
                   percent_rmse(y_test, ols_2_predictions)]

#MAE AND MAPE
from sklearn.metrics import mean_absolute_error
results['MAE'] = [mean_absolute_error(y_test, forest_1_predictions).round(2), 
                  mean_absolute_error(y_test, forest_2_predictions), 
                  mean_absolute_error(y_test, ols_1_predictions), 
                  mean_absolute_error(y_test, ols_2_predictions)]

def MAPE(y_true, y_pred):
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
    return mape

results['%MAE'] = [MAPE(y_test, forest_1_predictions),
                   MAPE(y_test, forest_2_predictions),
                   MAPE(y_test, ols_1_predictions),
                   MAPE(y_test, ols_2_predictions)]

#R2 (COEFFICIENT OF DETERMINATION)
from sklearn.metrics import r2_score
results['R-squared'] = [r2_score(y_test, forest_1_predictions), 
                  r2_score(y_test, forest_2_predictions), 
                  r2_score(y_test, ols_1_predictions), 
                  r2_score(y_test, ols_2_predictions)]

results = results.round(3)
results

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
for i in results.columns:
    plt.scatter(results.index, results[i], label=i)
plt.legend()
plt.xlabel('Model')
plt.ylabel('Performance')
plt.title('Final model performance')
plt.savefig('OUTPUT_models/model_performance.png')
plt.grid()
plt.show()

In [ ]:
#hit rates
y_test_for_hit_rate = list(y_test)
hit_rates = pd.DataFrame(index=['RF, all features', 'RF, 20 features', 'OLS, all features', 'OLS, 20 features'], 
                         columns=['Within 1%', 'Within 5%', 'Within 10%'])
predictions = [forest_1_predictions, forest_2_predictions, ols_1_predictions, ols_2_predictions]
def predict_hit_rate(y_true, y_pred, margin):
    a = 0
    rate_tmp = []
    for i in y_pred:
        if i >= y_true.iloc[a]*(1-margin):
            if i >= y_true.iloc[a]*(1+margin):
                rate_tmp.append(1)
            else:
                pass
        else:
            rate_tmp.append(0)
        a += 1
    return(sum(rate_tmp) / len(rate_tmp))

hit_rates['Within 1%']['RF, all features'] = predict_hit_rate(y_test, forest_1_predictions, 0.01)*100
hit_rates['Within 1%']['RF, 20 features'] = predict_hit_rate(y_test, forest_2_predictions, 0.01)*100
hit_rates['Within 1%']['OLS, all features'] = predict_hit_rate(y_test, ols_1_predictions, 0.01)*100
hit_rates['Within 1%']['OLS, 20 features'] = predict_hit_rate(y_test, ols_2_predictions, 0.01)*100
hit_rates['Within 5%']['RF, all features'] = predict_hit_rate(y_test, forest_1_predictions, 0.05)*100
hit_rates['Within 5%']['RF, 20 features'] = predict_hit_rate(y_test, forest_2_predictions, 0.05)*100
hit_rates['Within 5%']['OLS, all features'] = predict_hit_rate(y_test, ols_1_predictions, 0.05)*100
hit_rates['Within 5%']['OLS, 20 features'] = predict_hit_rate(y_test, ols_2_predictions, 0.05)*100
hit_rates['Within 10%']['RF, all features'] = predict_hit_rate(y_test, forest_1_predictions, 0.10)*100
hit_rates['Within 10%']['RF, 20 features'] = predict_hit_rate(y_test, forest_2_predictions, 0.10)*100
hit_rates['Within 10%']['OLS, all features'] = predict_hit_rate(y_test, ols_1_predictions, 0.10)*100
hit_rates['Within 10%']['OLS, 20 features'] = predict_hit_rate(y_test, ols_2_predictions, 0.10)*100

hit_rates.astype(float).round(3)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
for i in hit_rates.columns:
    plt.scatter(hit_rates.index, hit_rates[i], label=i)
plt.legend()
plt.xlabel('Model')
plt.ylabel('Percent of predictions with margin of actuals')
plt.title('Model hit rates')
plt.savefig('OUTPUT_models/hit_rates.png')
plt.grid()
plt.show()